In [4]:
import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup

In [24]:
from dotenv import load_dotenv
load_dotenv()

True

In [25]:
ds = pd.read_csv('../input/starbucks.csv', encoding="utf-8")

In [6]:
ds.shape

(25600, 13)

Hacemos scraping de countrycode.org para traernos el nombre del país a partir del codigo ISO del dataset

In [29]:
url_codes = 'https://countrycode.org/'
data_codes = requests.get(url_codes).text
soup_codes = BeautifulSoup(data_codes, 'html.parser')    

In [28]:
#[tag.find_all("th") for tag in soup_codes.find_all('table')][1]

In [36]:
def procesaFila(tag):
    m = tag.find_all("td")
    return {
        "pais":m[0].find("a").text.strip(),
        #"cod_n":m[1].text,
        "Country":m[2].text.split(' /')[0]
    }
    
table_codes = soup_codes.find_all('table')[1]
table_codes_clean = [procesaFila(fila) for fila in table_codes.find_all("tr")[1:]]

In [37]:
countries = pd.DataFrame(table_codes_clean)

Hacemos el merge de los dos dataframes

In [38]:
result = pd.merge(ds, countries, how='left', on=['Country', 'Country'])
result.head()